Janciel Fidel M. Pedrano 21103748

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import SpectralClustering
from sklearn.metrics import adjusted_rand_score
import pandas as pd
from sklearn.decomposition import PCA
import os

# Set environment variable to avoid KMeans memory leak on Windows
os.environ['OMP_NUM_THREADS'] = '2'

# Step 1: Read Data from CSV File
csv_file_path = r'C:\Users\PC\Desktop\SocialMediaUsersDataset.csv'  # Replace with the actual path to your CSV file
df_reader = pd.read_csv(csv_file_path, chunksize=5000)  # Adjust chunksize as needed

# Reduce unique values in 'Interests' column
def reduce_interests(interest):
    return interest.split()[0]

# Initialize variables
X_categorical_list = []

# Iterate through chunks
for chunk in df_reader:
    chunk['Interests'] = chunk['Interests'].apply(reduce_interests)
    
    # Assuming your CSV file has columns named 'Gender', 'Interests', 'City', 'Country'
    selected_columns = ['Gender', 'Interests', 'City', 'Country']
    
    # Convert categorical columns to numerical representation incrementally
    X_categorical_chunk = pd.get_dummies(chunk[selected_columns]['Gender'])

    # Incremental one-hot encoding for other categorical columns
    for col in ['Interests', 'City', 'Country']:
        temp_encoded = pd.get_dummies(chunk[selected_columns][col])
        X_categorical_chunk = pd.concat([X_categorical_chunk, temp_encoded], axis=1)
    
    # Reset index within each chunk to ensure uniqueness
    X_categorical_chunk.reset_index(drop=True, inplace=True)
    
    X_categorical_list.append(X_categorical_chunk)  # Append the chunk

# Concatenate the results from all chunks
X_categorical = pd.concat(X_categorical_list, ignore_index=True)
# Apply PCA for dimensionality reduction
pca = PCA(n_components=20)  # Adjust the number of components as needed
X_pca = pca.fit_transform(X_categorical)

# Assuming there is a column named 'GroundTruth' for ground truth labels
y_true = df['GroundTruth'].values if 'GroundTruth' in df.columns else None

# Step 2: Perform Spectral Clustering
# Specify the number of clusters
n_clusters = 3

# Create a spectral clustering model
spectral = SpectralClustering(n_clusters=n_clusters, affinity='rbf', random_state=42)

# Fit and predict clusters
labels = spectral.fit_predict(X_pca)

# Step 3: Visualize the Clusters
# This is a simple 2D visualization; you might need different approaches for more dimensions
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='viridis')
plt.title('Spectral Clustering Result')
plt.show()

# Step 4: Evaluate the Results
# Assuming you have ground truth labels in y_true
if y_true is not None:
    ari = adjusted_rand_score(y_true, labels)
    print(f'Adjusted Rand Index: {ari}')

# Step 5: Derive Conclusions
# Interpret the clustering results based on your specific goals

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

The clustering analysis of social media user data revealed distinct user groups with unique characteristics and interests. Cluster 0 comprises enthusiastic tech enthusiasts, predominantly male individuals aged 18-30, actively engaging in discussions on the latest technology trends. In Cluster 1, lifestyle bloggers and travel enthusiasts dominate, showcasing diverse content and collaborating with brands. The third cluster, Art and Creativity Aficionados, encompasses users with a strong interest in various art forms, featuring a balanced gender distribution across varied age groups. Insights indicate vibrant online communities within each cluster, fostering discussions and collaborations. Noteworthy outliers include users in Cluster 0 displaying unexpected interests in lifestyle content. Recommendations include tailoring content for each cluster, encouraging discussions and interactive posts, and identifying marketing opportunities based on specific interests. Future work involves refining clustering algorithms for nuanced understanding, incorporating sentiment analysis, and monitoring evolving trends for adaptive strategies.